In [18]:
import re
import os
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from pyspark.sql import Window
from functools import reduce

# CONSTANTS
HOST = 'output-db'
PORT = 5433
DATABASE = 'postgres'
USER = 'postgres'
PASSWORD = '1234'

INPUT_PATH = './notebooks/parquet/'
TABLE_NAMES = [
    'city',
]

In [19]:
spark= (
    SparkSession.builder.appName("write-to-db")
    .config("spark.jars", "/usr/local/spark/jars/postgresql.jar")    
    .getOrCreate()
)

_df_raw={}
for table_name in TABLE_NAMES:
    filename=rf'{INPUT_PATH}/{table_name}'
    _df_raw[table_name]=spark.read.format('parquet').option("header",True).load(filename)
    # _df_raw[table_name].printSchema()
    # _df_raw[table_name].show(10, truncate=False)

    # Write DataFrame to PostgreSQL
    jdbc_url = f'jdbc:postgresql://{HOST}:{PORT}/{DATABASE}'
    properties = {
        'user': f'{USER}',
        'password': f'{PASSWORD}',
        'driver': 'org.postgresql.Driver'
    }

    _df_raw[table_name].write.jdbc(url=jdbc_url, table=table_name, mode="overwrite", properties=properties)



Py4JJavaError: An error occurred while calling o140.jdbc.
: org.postgresql.util.PSQLException: Connection to output-db:5433 refused. Check that the hostname and port are correct and that the postmaster is accepting TCP/IP connections.
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:346)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:54)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:273)
	at org.postgresql.Driver.makeConnection(Driver.java:446)
	at org.postgresql.Driver.connect(Driver.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:160)
	at org.apache.spark.sql.jdbc.JdbcDialect.$ano